In [1]:
# pull in quality data from Wikimedia
from paws.EpochFail import wikiquality as wq

ImportError: No module named 'paws'

In [2]:
# check to make sure we got what we expected
print(next(wq.read_rows('en')))

{'weighted_sum': 0.10413085831157592, 'timestamp': '20051201000000', 'rev_id': 28847753, 'prediction': 'Stub', 'title': 'Lost Moon', 'page_id': 3204489}


In [3]:
# here's what those 'predicted' values mean: 
# https://meta.wikimedia.org/wiki/Objective_Revision_Evaluation_Service#Article_quality_models


In [4]:
# look for the women scientists
print(next(wq.read_page_ids('women scientists')))

{'page_id': 9622178}
